In [12]:
!pip install langchain==0.2.14
!pip install langchain-openai==0.1.8
!pip install langchainhub==0.1.20
!pip install tiktoken==0.7.0

In [13]:
import os
import getpass

# # Set OPENAI API Key

os.environ["OPENAI_API_KEY"] = getpass.getpass()

# OR (load from .env file)

# from dotenv import load_dotenv
# load_dotenv("./.env")

The more complex version of this would use this pygithub package as shown in here:
- https://python.langchain.com/docs/integrations/toolkits/github

But let's try a simpler version that just sets up a bunch of python functions to automate different github actions like add, commit and pull requests.

Let's start with a simple commit to a branch of some repository.

First, I'll create a github repository using some simple commands.

In [14]:
# git init -b main

# git add . && git commit -m "Some commit"

# gh repo create

In [1]:
MODEL='gpt-4o-mini'
TEMP=0.0

Now, let's try asking an agent to write come code and commit the resulting code to the current branch of this repository.

To do that, let's give the agent the necessary tools it will need which in this case will be python functions that perform different github actions using the `subprocess` package.

We'll follow the basic steps for building a langchain agent:

# Steps for building a simple agent:
- Set up the LLM
- Define the tool or toolkit (list of tools)
- Set up a prompt template
- Connect llm with your tools
- Define your agent as a dict with keys: input, and agent scratchpad 
- Use the Langchain LCEL language to pipe agent, prompt the llm_with_tools variable and the output parser (which can use OpenAI function calling parser)
- Create the agent loop
- Wrap everything into the AgentExecutor
- Invoke the agent with some query input


In [2]:
# setup the llm
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=MODEL, temperature=TEMP)

Now, let's create a tool for the LLM

In [3]:
import subprocess

def github_commit_tool(commit_message="Some commit"):
    subprocess.run(["git", "add", "."])
    subprocess.run(["git", "commit", "-m", commit_message])
    subprocess.run(["git", "push", "-u", "origin", "main"])
    
    return "Committed to Github"

Now! Before we use it with langchain, let's test it by itself.

In [22]:
github_commit_tool("Some testing commit")

[main ae43492] Some testing commit
 2 files changed, 86 insertions(+), 71 deletions(-)
branch 'main' set up to track 'origin/main'.


To https://github.com/EnkrateiaLucca/oreilly_live_training_getting_started_with_langchain.git
   b572795..ae43492  main -> main


'Committed to Github'

Now, let's take a look at out github repository from the terminal.

![](2023-10-29-20-47-54.png)

Nice! It looks like we are good to go with this first simple tool!

Let's now make it a tool for our langchain agent by adding the @tool. 

In [3]:
from langchain.tools import tool

@tool
def github_commit_tool(commit_message="Some commit"):
    """This function uses the subprocess package to make commits to a github repo pre-defined."""
    subprocess.run(["git", "add", "."])
    subprocess.run(["git", "commit", "-m", commit_message])
    subprocess.run(["git", "push", "-u", "origin", "main"])
    
    return "Committed to Github"

tools = [github_commit_tool]

We added some documentation to our function to abide by the requirements of the tool decorator from langchain.

Now, let's test if a simple agent can use that tool!

Let's start by setting up our prompt template.

In [20]:
# from langchain.agents import AgentExecutor
# from langchain.prompts import ChatPromptTemplate

# template = """
# You are a helpful programming assistant. The user will ask you to perform commits to a github repo and you will perform the needed actions and call the necessary
# functions when needed.
# Your current task:
# {input}
# """
# prompt = ChatPromptTemplate.from_template(template)

In [4]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [5]:
from langchain.agents import create_openai_tools_agent
from langchain_openai import ChatOpenAI

llm_chat = ChatOpenAI(temperature=0)
agent = create_openai_tools_agent(llm_chat, tools, prompt)

In [6]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent,tools=tools, verbose=True)

In [7]:
agent_executor.invoke({"input": "Create a github commit with the message 'Testing openai tools agent!'"})



> Entering new AgentExecutor chain...

Invoking: `github_commit_tool` with `{'commit_message': 'Testing openai tools agent!'}`


[main 45844f5] Testing openai tools agent!
 7 files changed, 340 insertions(+), 321 deletions(-)


To https://github.com/EnkrateiaLucca/oreilly_live_training_getting_started_with_langchain.git
   28bc4f4..45844f5  main -> main


branch 'main' set up to track 'origin/main'.
Committed to GithubI have successfully created a GitHub commit with the message 'Testing openai tools agent!'.

> Finished chain.


{'input': "Create a github commit with the message 'Testing openai tools agent!'",
 'output': "I have successfully created a GitHub commit with the message 'Testing openai tools agent!'."}

And voila, the agent calls the right function and creates the commit!

(ignore the github related error, let's focus on the agent capabilities).

And voila! We made a commit!

![](2023-10-30-13-04-20.png)

Ok, this is cool but, we can go beyond that, and add the ability to create a file and then commit that file to our desired repository. Let's do that!

To do it all we have to do is amplify the Agent's toolkit with another tool that can create files in the current folder.

Perfect! Now, let's add this to our github agent

In [8]:
@tool
def create_file_tool(file_contents):
    """This function creates a file with the contents and file path provided as inputs."""
    with open("file.py", "w") as f:
        f.write(file_contents)
    
    return "File created"

Now, let's add that functionality to our agent by just updating the tools list.

In [9]:
tools = [github_commit_tool, create_file_tool]

Now, let's modify our agent with its new toolkit

In [10]:
agent = create_openai_tools_agent(llm_chat, tools, prompt)

In [11]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

Now, let's try using both tools in a call to see if the agent can do something slightly more complex now.

In [12]:
agent_executor.invoke({
    "input": "Write Python code with a for loop stating the truth about what is better: pancakes or waffles.\
    Output ONLY THE CODE and save that to a file then commit it to our github repository."
})



> Entering new AgentExecutor chain...

Invoking: `create_file_tool` with `{'file_contents': "# Python code\n\ntruths = ['Pancakes are better than waffles.', 'Waffles are better than pancakes.']\n\nfor truth in truths:\n    print(truth)\n", 'file_path': 'pancakes_vs_waffles.py'}`


File created
Invoking: `github_commit_tool` with `{'commit_message': 'Add Python code comparing pancakes and waffles'}`


[main fdd3f5b] Add Python code comparing pancakes and waffles
 2 files changed, 49 insertions(+), 48 deletions(-)
 create mode 100644 notebooks/file.py


To https://github.com/EnkrateiaLucca/oreilly_live_training_getting_started_with_langchain.git
   45844f5..fdd3f5b  main -> main


branch 'main' set up to track 'origin/main'.
Committed to GithubI have written the Python code comparing pancakes and waffles, saved it to a file, and committed it to our GitHub repository.

> Finished chain.


{'input': 'Write Python code with a for loop stating the truth about what is better: pancakes or waffles.    Output ONLY THE CODE and save that to a file then commit it to our github repository.',
 'output': 'I have written the Python code comparing pancakes and waffles, saved it to a file, and committed it to our GitHub repository.'}

And voila again!
![](2023-10-30-13-15-38.png)

And again we succeed!

![](2023-10-30-13-26-32.png)

Ok, before we finish with this example, let's add a last tool to read files from a repo, for situations where we might write some Python code and we want our agent to integrate that Python code into our repository. 

Again, we follow the same procedure:

- Create the tool
- Update our tookit
- Update our agent
- Run a test task to check if it works 

![](2023-10-30-13-42-25.png)

I think this is a great usecase for agents like these even if sometimes the performance and reliability is a bit off, because now, its a matter of optimizing the edges to get it to work for complex use cases.